In [3]:
# Replicate ITI results, make sure ITI utils and probing utils work right

#%%
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the TransformerLens code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")
    
import plotly.io as pio
# pio.renderers.default = "png"
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

from tqdm import tqdm
from utils.probing_utils import ModelActs
from utils.dataset_utils import CounterFact_Dataset, TQA_MC_Dataset, EZ_Dataset

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from utils.iti_utils import patch_iti

from utils.analytics_utils import plot_probe_accuracies, plot_norm_diffs, plot_cosine_sims

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_233912/1448159700.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_233912/1448159700.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [4]:

device = "cuda"
print("loading model")
model = HookedTransformer.from_pretrained(
    "gpt2-xl",
    center_unembed=False,
    center_writing_weights=False,
    fold_ln=False,
    refactor_factored_attn_matrices=True,
    device=device,
)
# model.to(device)
print("done")
model.set_use_attn_result(True)
model.cfg.total_heads = model.cfg.n_heads * model.cfg.n_layers

model.reset_hooks()

loading model


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
done


In [5]:
random_seed = 5

datanames = ["tqa", "cfact", "ez"]

tqa_data = TQA_MC_Dataset(model.tokenizer, seed=random_seed)
cfact_data = CounterFact_Dataset(model.tokenizer, seed=random_seed)
ez_data = EZ_Dataset(model.tokenizer, seed=random_seed)

datasets = {"tqa":tqa_data, "cfact":cfact_data, "ez":ez_data}

Found cached dataset truthful_qa (/root/.cache/huggingface/datasets/truthful_qa/multiple_choice/1.1.0/63502f6bc6ee493830ce0843991b028d0ab568d221896b2ee3b8a5dfdaa9d7f4)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/root/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--counterfact-tracing-39c4f800d46af5cf/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-7c941ef28bd7dd5b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
n_acts = 1000
acts = {}

for name in datanames:
    acts[name] = ModelActs(model, datasets[name], act_types=["z", "mlp_out", "resid_post", "resid_pre", "result"])
    model_acts: ModelActs = acts[name]
    model_acts.gen_acts(N=n_acts, id=f"{name}_gpt2xl_{n_acts}")
    # break
    # ez_acts.load_acts(id=f"ez_gpt2xl_{n_acts}", load_probes=False)
    model_acts.train_probes("z", max_iter=1000)


100%|██████████| 1000/1000 [01:12<00:00, 13.83it/s]


Stored at tqa_gpt2xl_1000
torch.Size([800, 1200, 64]), torch.Size([200, 1200, 64]), torch.Size([800, 1200]), torch.Size([200, 1200])


100%|██████████| 1200/1200 [00:09<00:00, 122.73it/s]
/root/iti_capstone/utils/dataset_utils.py:156: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return indices, np.array(self.all_prompts)[indices], np.array(self.all_labels)[indices]
/root/iti_capstone/utils/dataset_utils.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return indices, np.array(self.all_prompts

KeyboardInterrupt: 

In [13]:
from utils.gpt_judge import get_iti_scores
plots = []

for name in datanames:
    get_iti_scores(model, datasets[name], alpha=20, topk=50)

100%|██████████| 1000/1000 [01:04<00:00, 15.41it/s]


Stored at gpt2xl_1000
torch.Size([800, 1200, 64]), torch.Size([200, 1200, 64]), torch.Size([800, 1200]), torch.Size([200, 1200])


100%|██████████| 1200/1200 [00:10<00:00, 117.21it/s]


truth_score=0.66, info_score=0.72, truth_score_iti=0.94, info_score_iti=0.26


/root/iti_capstone/utils/dataset_utils.py:156: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return indices, np.array(self.all_prompts)[indices], np.array(self.all_labels)[indices]
/root/iti_capstone/utils/dataset_utils.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return indices, np.array(self.all_prompts)[indices], np.array(self.all_labels)[indices]
100%|█

...

truth_score=0.9, info_score=0.34, truth_score_iti=0.96, info_score_iti=0.12


100%|██████████| 1000/1000 [01:04<00:00, 15.43it/s]


Stored at gpt2xl_1000
torch.Size([800, 1200, 64]), torch.Size([200, 1200, 64]), torch.Size([800, 1200]), torch.Size([200, 1200])


100%|██████████| 1200/1200 [00:09<00:00, 123.93it/s]


truth_score=0.62, info_score=0.64, truth_score_iti=1.0, info_score_iti=0.02


In [14]:
from utils.gpt_judge import get_iti_scores
plots = []

for name in datanames:
    get_iti_scores(model, datasets[name], alpha=1, topk=1200)

100%|██████████| 1000/1000 [01:04<00:00, 15.48it/s]


Stored at gpt2xl_1000
torch.Size([800, 1200, 64]), torch.Size([200, 1200, 64]), torch.Size([800, 1200]), torch.Size([200, 1200])


100%|██████████| 1200/1200 [00:25<00:00, 46.49it/s] 


truth_score=0.52, info_score=0.7, truth_score_iti=0.74, info_score_iti=0.68


/root/iti_capstone/utils/dataset_utils.py:156: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return indices, np.array(self.all_prompts)[indices], np.array(self.all_labels)[indices]
/root/iti_capstone/utils/dataset_utils.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return indices, np.array(self.all_prompts)[indices], np.array(self.all_labels)[indices]
100%|█

...

truth_score=0.9, info_score=0.28, truth_score_iti=0.82, info_score_iti=0.44


100%|██████████| 1000/1000 [01:04<00:00, 15.50it/s]


Stored at gpt2xl_1000
torch.Size([800, 1200, 64]), torch.Size([200, 1200, 64]), torch.Size([800, 1200]), torch.Size([200, 1200])


100%|██████████| 1200/1200 [00:09<00:00, 120.51it/s]


truth_score=0.74, info_score=0.66, truth_score_iti=0.7, info_score_iti=0.74


In [9]:
from plotly.subplots import make_subplots
from utils.gpt_judge import check_iti_generalization
plots = []

for name in datanames:
    model_acts: ModelActs = acts[name]
    for other_name in datanames:
        print(f"Checking generation on {name}, ITI on {other_name}")
        results = check_iti_generalization(model, datasets[name], datasets[other_name], 50, 1000, alpha=1, topk=1200, existing_gen_acts=model_acts)
        print(f"Truth score before ITI: {results[0]}, Truth score after ITI: {results[2]}")
        print(f"Info score before ITI: {results[1]}, Info score after ITI: {results[3]}")
        print()

        break
    break

        # transfer_accs = model_acts.get_transfer_acc(acts[other_name])
        # plots.append(plot_probe_accuracies(model_acts, sorted=False, title=f"{name} probes on {other_name} data", other_head_accs=transfer_accs).show())

Checking generation on tqa, ITI on tqa
truth_score=0.58, info_score=0.7, truth_score_iti=0.7, info_score_iti=0.7
Truth score before ITI: 0.58, Truth score after ITI: 0.7
Info score before ITI: 0.7, Info score after ITI: 0.7



In [14]:
from plotly.subplots import make_subplots
from utils.gpt_judge import check_iti_generalization
plots = []

np.seterr(all="ignore")
for name in datanames[1:]:
    model_acts: ModelActs = acts[name]
    for other_name in datanames:
        print(f"Checking generation on {name}, ITI on {other_name}")
        results = check_iti_generalization(model, datasets[name], datasets[other_name], 50, 1000, alpha=10, existing_gen_acts=acts[name])
        print(f"Truth score before ITI: {results[0]}, Truth score after ITI: {results[2]}")
        print(f"Info score before ITI: {results[1]}, Info score after ITI: {results[3]}")
        print()

        # transfer_accs = model_acts.get_transfer_acc(acts[other_name])
        # plots.append(plot_probe_accuracies(model_acts, sorted=False, title=f"{name} probes on {other_name} data", other_head_accs=transfer_accs).show())

Checking generation on cfact, ITI on tqa
truth_score=0.84, info_score=0.32, truth_score_iti=0.92, info_score_iti=0.3
Truth score before ITI: 0.84, Truth score after ITI: 0.92
Info score before ITI: 0.32, Info score after ITI: 0.3

Checking generation on cfact, ITI on cfact
truth_score=0.92, info_score=0.36, truth_score_iti=0.9, info_score_iti=0.32
Truth score before ITI: 0.92, Truth score after ITI: 0.9
Info score before ITI: 0.36, Info score after ITI: 0.32

Checking generation on cfact, ITI on ez
truth_score=0.8, info_score=0.52, truth_score_iti=0.92, info_score_iti=0.26
Truth score before ITI: 0.8, Truth score after ITI: 0.92
Info score before ITI: 0.52, Info score after ITI: 0.26

Checking generation on ez, ITI on tqa
truth_score=0.68, info_score=0.66, truth_score_iti=0.78, info_score_iti=0.6
Truth score before ITI: 0.68, Truth score after ITI: 0.78
Info score before ITI: 0.66, Info score after ITI: 0.6

Checking generation on ez, ITI on cfact
truth_score=0.74, info_score=0.6, trut

In [ ]:
import plotly.graph_objects as go

fig_combined = make_subplots(rows=3, cols=3)

for i, fig in enumerate(plots):
    row = i // 3 + 1  # calculate the row index
    col = i % 3 + 1   # calculate the column index

    # Extract data from the individual figures and add it to the subplots
    for trace in fig.data:
        fig_combined.add_trace(
            go.Heatmap(
                z=trace.z,
                x0=trace.x[0],
                dx=trace.x[1] - trace.x[0],
                y0=trace.y[0],
                dy=trace.y[1] - trace.y[0],
                zmin=trace.zmin,
                zmax=trace.zmax,
                coloraxis=trace.coloraxis,
                showscale=False,
            ),
            row=row,
            col=col,
        )

# Add a colorbar that's common to all subplots
fig.update_layout(coloraxis=dict(colorscale='viridis', colorbar=dict(tickfont=dict(size=10))))

fig.show()

In [ ]:
plots